- https://foofish.net/python-decorator.html
- 2019/08/30

In [1]:
### python 中的 function 為 一級函式, 可以當作參數, 傳送給其他 function 當作 參數
def foo():
    print('foo')
    
def bar(func):
    func()
    
bar(foo)

foo


> decorator 本質上是一個 `python function / python class`. decorator 的 return value 其實就是個 `function/object`

> ex: logging, 性能測試, transaction, cache, verify auth... 等


## 底下一個範例, 示範 decorator 改寫過程, 分為 4 步

In [2]:
import logging
logging.basicConfig(level=logging.DEBUG)

### 1

In [3]:
### 大量重複
import logging

def foo():
    print('this is foo')
    logging.debug('foo is running')

def foo2():
    print('this is foo2')
    logging.debug('foo2 is running')

def foo3():
    print('this is foo3')
    logging.debug('foo3 is running')

foo2()

DEBUG:root:foo2 is running


this is foo2


### 2

In [4]:
### 減少程式碼了, 但似乎更糟糕. 把程式邏輯 && logging 邏輯混合在一起
import logging

def use_loggin(func):
    logging.debug(f'{func.__name__} is running')
    func()

def foo():
    print('this is foo')

def foo2():
    print('this is foo2')

def foo3():
    print('this is foo3')

use_loggin(foo2)

DEBUG:root:foo2 is running


this is foo2


### 3. 改寫為 decorator 半套的寫法

In [6]:
import logging

def use_logging(func):
    def business():
        logging.debug(f'{func.__name__} is running')
        return func()
    return business

def foo():
    print('this is foo')

def foo2():
    print('this is foo2')

def foo3():
    print('this is foo3')

foo2 = use_logging(foo2)
foo2()

DEBUG:root:foo2 is running


this is foo2


### 4. decorator 完整寫法

有了 「@ 語法蜜糖」, 省略掉了 foo2 = use_logging(foo2) 這種寫法

function return function, 即為 decorator. 裏頭包裹著業 `務邏輯函式(func)`

In [7]:
import logging

def use_logging(func):
    def business():
        logging.debug(f'{func.__name__} is running')
        return func()
    return business

@use_logging
def foo():
    print('this is foo')

@use_logging
def foo2():
    print('this is foo2')

@use_logging
def foo3():
    print('this is foo3')

foo2()

DEBUG:root:foo2 is running


this is foo2


## decorator + paramaters

### 1. 一個參數

業務邏輯 foo 們需要帶參數

In [8]:
import logging

def use_logging(func):
    def business(msg):
        logging.debug(f'{func.__name__} is running')
        return func(msg)
    return business

@use_logging
def foo(msg):
    print(msg)

@use_logging
def foo2(msg):
    print(msg)

@use_logging
def foo3(msg):
    print(msg)

foo2(55)

DEBUG:root:foo2 is running


55


### 2. 很多很多參數, 使用 \*args, \**kwargs

In [9]:
import logging

def use_logging(level):
    def decorator(func):
        def business(*args, **kwargs):
            if level == 'debug':
                logging.debug(f'{func.__name__} is running')
            elif level == 'info':
                logging.info(f'{func.__name__} is running')
            elif level == 'warn':
                logging.warn(f'{func.__name__} is running')
            return func(*args, **kwargs)
        return business
    return decorator

@use_logging(level='debug')
def foo(msg):
    print(msg)

@use_logging(level='info')
def foo2(msg):
    print(msg)

@use_logging(level='warn')
def foo3(msg):
    print(msg)

foo2(55)

INFO:root:foo2 is running


55


## class decoarator

主要依靠 \_\_call__ , 使用「@」可在 decorator 附加到 function 之上

In [17]:
from dataclasses import dataclass

@dataclass
class Foo:
    func: any
        
    def __call__(self):
        print('class decorator start')
        self.func()
        print('class decorator end')
      
@Foo
def bar():
    print('bar')

bar()

class decorator start
bar
class decorator end
